In [ ]:
%reload_ext autoreload
%autoreload 2

from Simulator import Simulator
from BinRealGenerator import BinRealGenerator
import numpy as np
import utils
from matplotlib import pyplot as plt
from HistoryMatching import HistoryMatching

#Данные, полученные по snesim.py (realsmatrix.npy),  Training.ipynb (parameterizing_saved_model)
realgen = BinRealGenerator("test_data/realsmatrix.npy", "test_data/parameterizing_saved_model", 170, sand_perm_val = 2000, mud_perm_val=200, U_cache_file =  "test_data/U.npy", S_cache_file = "test_data/S.npy", Vh_cache_file = "test_data/Vh.npy")
#Egg_Model_ECL - статические параметры гидродинамического симулятора
sim = Simulator("test_data/models/EGG", "Egg_Model_ECL.DATA", "PermiabilityMap.INC", "PorosityMap.INC", "EGG_MODEL_ECL.UNSMRY", threads_count=50, init_dirs=True)


Тестовая модель

In [ ]:
test_reals = np.load("test_data/test_reals.npy")
source = test_reals[193][:,:,0].T

permiabilities = source.copy()
permiabilities[permiabilities==0] = 200
permiabilities[permiabilities==1] = 2000
porosities = source.copy().astype(float)
porosities[porosities==0]=0.15
porosities[porosities==1]=0.25

injectors = [(18, 90), (40, 90), (2, 75), (2, 60), (2,25)]

producers = [(90, 40), (53, 40), (58, 2), (40, 2)]
plt.imshow(permiabilities)
utils.plotWells(plt, injectors, producers)

observed = sim.run_simulator_source_dir(permiabilities, porosities)

years_obs = observed.numpy_vector("YEARS", report_only=True)
fopr_obs = observed.numpy_vector("FOPR", report_only=True) 
noise_const = 0
observed_data = [fopr_obs + noise_const + np.random.normal(size = len(years_obs), scale=0.03 * abs(fopr_obs.max()+1))]
for x in range(1,5):
    wellInfo = observed.numpy_vector(f"WOPR:PROD{x}", report_only=True)
    observed_data.append(wellInfo + np.random.normal(scale = 0.03 * abs(wellInfo.max()+1), size = len(wellInfo)))

In [ ]:
prior = np.random.normal(size = 170)
matcher = HistoryMatching(realgen, sim)
matcher.match_model(prior, observed_data)